## Configuration

In [1]:
### Configuration
# Import dependencies
import os
from collections import Counter
import pandas as pd
from summerplaylist import *

# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

## Log In

In [3]:
# Authenticate to Spotify
credentials = authenticateToSpotify(CLIENTID, CLIENTSECRET, REDIRECT_URI)

Opening browser for Spotify login...


## Read in current song candidates

In [4]:
# Get playlist IDs
playlistID1 = getPlaylistID(credentials, '2023')
playlistID2 = getPlaylistID(credentials, '2023 Round 2')

In [7]:
# Read in CurrentYearSongCandidates playlist
playlistTracks1 = fetchTracksFromPlaylist(credentials, playlistID1)
playlistTracks2 = fetchTracksFromPlaylist(credentials, playlistID2)
# Merge dfs
allNewTracks = pd.concat([playlistTracks1, playlistTracks2], axis=0, ignore_index=True)
allNewTracks

,artistID,trackName,release_date,songURI
0,12Zk1DFhCbHY6v3xep2ZjI,Skin and Bones,2022-06-03,spotify:track:4CZgQq8KJYWGh1bM3sqGmd
1,12Zk1DFhCbHY6v3xep2ZjI,Wine & Spirits,2022-06-03,spotify:track:02iwkyyf6rSZPAe0NQOcjL
2,12Zk1DFhCbHY6v3xep2ZjI,Come Back Home,2022-06-03,spotify:track:1jiOVoiDZdtHiaF5W9XqWN
3,5jQsMNuCW0iZeFTz6YUK8K,Claw Your Way Out,2022-01-27,spotify:track:3HjohYDsYZg2KLhYjto8Hq
4,2f0NSj1t2L6JowHINXCFb6,No Feeling Is Forever,2022-06-10,spotify:track:68V71NzNC6GRaCaE9S8bcO
...,...,...,...,...
268,1X6AU9VgQPke2P4ES0EhJ3,keepsafe (feat. Laeland),2022-12-02,spotify:track:3hsXQNVVzLc5UgwNqEeY19
269,6bmlMHgSheBauioMgKv2tn,broke my habits,2022-12-02,spotify:track:3mwwnrQ2T9Yv8gACR1c7aQ
270,6bmlMHgSheBauioMgKv2tn,watch me miss (feat. Jomie),2022-12-02,spotify:track:3oBnmomDWSl14PnTktpeNN
271,5F4cFdG9NvJMnHXJkmqw9g,watch me miss (feat. Jomie),2022-12-02,spotify:track:3oBnmomDWSl14PnTktpeNN


In [16]:
def fetchTracksFromPlaylist(credentials, playlistID):
    spotify, userID, user_token = credentials
    offset = 0
    limit = 100
    nestedList = ["empty"]
    result = pd.DataFrame()
    while len(nestedList) > 0:
        # Get all artists in the list from spotify
        nestedList = [[{
            "artistID":y.id, "trackName":x.track.name, "release_date":x.track.album.release_date, "songURI":x.track.uri
            } for y in x.track.artists
            ] for x in spotify.playlist_items(playlistID, offset=offset, limit=limit).items]
        # Unnest the list 
        unnestedList = [item for sublist in nestedList for item in sublist]
        toAdd = pd.DataFrame(unnestedList)
        result = pd.concat([result, toAdd], axis=0, ignore_index=True)
        offset += 100
    return result
fetchTracksFromPlaylist(credentials, playlistID1)

,artistID,trackName,release_date,songURI
0,12Zk1DFhCbHY6v3xep2ZjI,Skin and Bones,2022-06-03,spotify:track:4CZgQq8KJYWGh1bM3sqGmd
1,12Zk1DFhCbHY6v3xep2ZjI,Wine & Spirits,2022-06-03,spotify:track:02iwkyyf6rSZPAe0NQOcjL
2,12Zk1DFhCbHY6v3xep2ZjI,Come Back Home,2022-06-03,spotify:track:1jiOVoiDZdtHiaF5W9XqWN
3,5jQsMNuCW0iZeFTz6YUK8K,Claw Your Way Out,2022-01-27,spotify:track:3HjohYDsYZg2KLhYjto8Hq
4,2f0NSj1t2L6JowHINXCFb6,No Feeling Is Forever,2022-06-10,spotify:track:68V71NzNC6GRaCaE9S8bcO
...,...,...,...,...
442,6erYbDfpqSME4gIPCzf3i4,Glass Top Roof (The One),2023-01-26,spotify:track:0M1FSdIxVunVsciodnmYcn
443,6C403AR4y6PjN0xNNGh42m,Glass Top Roof (The One),2023-01-26,spotify:track:0M1FSdIxVunVsciodnmYcn
444,2DaP4uXwKOXAaD77XokW9a,Hasta La Vista,2020-07-03,spotify:track:4YHE1JU65JhFMzkHqKuOu8
445,0RZewmdiKrFtejSJ3jfmGs,Muay Thai,2023-03-22,spotify:track:31Z7QbuOg3Ljv0vZI6f0eR


In [8]:
playlistTracks1

,artistID,trackName,release_date,songURI
0,12Zk1DFhCbHY6v3xep2ZjI,Skin and Bones,2022-06-03,spotify:track:4CZgQq8KJYWGh1bM3sqGmd
1,12Zk1DFhCbHY6v3xep2ZjI,Wine & Spirits,2022-06-03,spotify:track:02iwkyyf6rSZPAe0NQOcjL
2,12Zk1DFhCbHY6v3xep2ZjI,Come Back Home,2022-06-03,spotify:track:1jiOVoiDZdtHiaF5W9XqWN
3,5jQsMNuCW0iZeFTz6YUK8K,Claw Your Way Out,2022-01-27,spotify:track:3HjohYDsYZg2KLhYjto8Hq
4,2f0NSj1t2L6JowHINXCFb6,No Feeling Is Forever,2022-06-10,spotify:track:68V71NzNC6GRaCaE9S8bcO
...,...,...,...,...
135,07YZf4WDAMNwqr4jfgOZ8y,Acapulco,2021-09-03,spotify:track:3eJH2nAjvNXdmPfBkALiPZ
136,6wWVKhxIU2cEi0K81v7HvP,Ausländer,2019-05-17,spotify:track:7j43FohbLVulScL7S9sQZk
137,3JhNCzhSMTxs9WLGJJxWOY,NO BAD DAYS (feat. Collett),2023-03-03,spotify:track:5C8ySsx3AT121g24uYR823
138,6u079TgFSFrxtv8iqq0Zn9,NO BAD DAYS (feat. Collett),2023-03-03,spotify:track:5C8ySsx3AT121g24uYR823


In [9]:
playlistTracks2

,artistID,trackName,release_date,songURI
0,5CuU6SRJjbbZL926nSGGxX,79th and Stony Island,2022-10-07,spotify:track:41t7RkFBjaEFoGvr29QhYL
1,5CuU6SRJjbbZL926nSGGxX,Burner Account,2022-10-07,spotify:track:2c9ZDyosWJkguChcMJAV8K
2,3SCI7sXHUZIeDKMWC5NT9C,Burner Account,2022-10-07,spotify:track:2c9ZDyosWJkguChcMJAV8K
3,5CuU6SRJjbbZL926nSGGxX,Multi-Game Arcade Cabinet,2022-10-07,spotify:track:46lQe6895zxB3HRP6YPbnl
4,1TZWbbk7dpKRVKE1EUs5Bt,Multi-Game Arcade Cabinet,2022-10-07,spotify:track:46lQe6895zxB3HRP6YPbnl
...,...,...,...,...
128,1X6AU9VgQPke2P4ES0EhJ3,keepsafe (feat. Laeland),2022-12-02,spotify:track:3hsXQNVVzLc5UgwNqEeY19
129,6bmlMHgSheBauioMgKv2tn,broke my habits,2022-12-02,spotify:track:3mwwnrQ2T9Yv8gACR1c7aQ
130,6bmlMHgSheBauioMgKv2tn,watch me miss (feat. Jomie),2022-12-02,spotify:track:3oBnmomDWSl14PnTktpeNN
131,5F4cFdG9NvJMnHXJkmqw9g,watch me miss (feat. Jomie),2022-12-02,spotify:track:3oBnmomDWSl14PnTktpeNN


In [ ]:
# Sort by frequency of artist, artist


## Filter out non-qualifiers

In [1]:
# Read in all old playlists
# Filter out any songs that have already appeared in old playlists
# Filter out any songs older than 5 years